In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
from scipy.stats import ttest_ind
import numpy as np
import seaborn as sns
import measurements as m
from core import Experiment, Rat, TrialEpoch
from plotting import plot_overtime, plot_behavior, add_col

In [ ]:
this_session = ['!2017-10-19']

In [ ]:
counterconditioning = ['!2017-10-04', '!2017-10-05', '!2017-10-06',
                       '!2017-10-07', '!2017-10-08', '!2017-10-09', 
                       '!2017-10-10', '!2017-10-11', '!2017-10-12',
                       '!2017-10-13', '!2017-10-14', '!2017-10-15',
                       '!2017-10-16', '!2017-10-17', '!2017-10-18',
                       '!2017-10-19']

In [ ]:
probe_sessions = ['!2017-10-16', '!2017-10-17', '!2017-10-18', '!2017-10-19']

In [ ]:
probe_colours = {'baseline, ':'#252525', 'light, rewarded':'#1f77b4', 'light, unrewarded':'#aec7e8', 
                 'probe, rewarded':'#d62728', 'probe, unrewarded':'#ff9896'}
overtime_colours = {'light1':'#9970ab', 'light2':'#5aae61'}

In [ ]:
for r, g, s in zip(['R155', 'R156', 'R157', 'R158', 'R159', 'R160', 'R161', 'R162'], 
                [1, 2, 2, 1, 2, 1, 1, 2], 
                ['male', 'female', 'male', 'female', 'male', 'female', 'male', 'female']):
    rat = [Rat(r, group=g, gender=s)]

    binned_expt = Experiment(
        name="201709",
        cache_key="binned",
        trial_epochs=[
            TrialEpoch("mags", start_idx=1, stop_idx=2),
            TrialEpoch("light1", start_idx=4, stop_idx=5),
            TrialEpoch("light2", start_idx=6, stop_idx=7),
            TrialEpoch("prelight1", start_idx=4, duration=-20),
            TrialEpoch("prelight2", start_idx=6, duration=-20),
        ],
        measurements=[m.Duration(), m.Count()],
        rats=rat,
        ignore_sessions=['!2017-09-20'],
        sessionfiles=this_session
    )


    def add_datapoints(session, data, rat):
        session.add_binned_data(rat.rat_id, data["light1"], binsize=5, info={'cue': 'light1'})
        session.add_binned_data(rat.rat_id, data["light2"], binsize=5, info={'cue': 'light2'})

    binned_expt.add_datapoints = add_datapoints
    binned_df = binned_expt.analyze(cached_data=False)
    binned_df = binned_df.drop(binned_df.loc[(binned_df.duration == 155) &
                                             (binned_df.time_start == 150)].index)
    binned_df = binned_df.drop(binned_df.loc[(binned_df.duration == 185) &
                                             (binned_df.time_start == 180)].index)
    binned_df = binned_df.drop(binned_df.loc[(binned_df.duration == 215) &
                                             (binned_df.time_start == 210)].index)
    binned_df = binned_df.drop(binned_df.loc[(binned_df.duration == 245) &
                                             (binned_df.time_start == 240)].index)
    binned_df = binned_df.drop(binned_df.loc[(binned_df.duration == 275) &
                                             (binned_df.time_start == 270)].index)

    binned_df.loc[binned_df.duration == 155, 'duration'] = 150
    binned_df.loc[binned_df.duration == 185, 'duration'] = 180
    binned_df.loc[binned_df.duration == 215, 'duration'] = 210
    binned_df.loc[binned_df.duration == 245, 'duration'] = 240
    binned_df.loc[binned_df.duration == 275, 'duration'] = 270

    print(rat[0].rat_id, "group:", rat[0].group)

    # duration t-test
    duration_df = binned_df[binned_df['measure']=='Duration']
    dur1 = duration_df[duration_df['cue']=='light1']
    dur2 = duration_df[duration_df['cue']=='light2']

    print('Duration:', ttest_ind(dur1['value'], dur2['value']))
    print('light1:', dur1['value'].mean())
    print('light2:', dur2['value'].mean())

    plot_overtime(binned_df, rats=rat, filepath=None, measure="Duration",
                  labels="Duration in food cup (s)", colours=overtime_colours)

    # count t-test
    count_df = binned_df[binned_df['measure']=='Count']
    count1 = count_df[count_df['cue']=='light1']
    count2 = count_df[count_df['cue']=='light2']

    print('Count:', ttest_ind(count1['value'], count2['value']))
    print('light1:', count1['value'].mean())
    print('light2:', count2['value'].mean())

    plot_overtime(binned_df, rats=rat, filepath=None, measure="Count", 
                  labels="Number of entries", colours=overtime_colours)

In [ ]:
epoch_expt = Experiment(
    name="201709",
    cache_key="epoch",
    plot_key="",
    trial_epochs=[
        TrialEpoch("mags", start_idx=1, stop_idx=2),
        TrialEpoch("light1", start_idx=4, stop_idx=5),
        TrialEpoch("light2", start_idx=6, stop_idx=7),
        TrialEpoch("trial1", start_idx=12, stop_idx=13),
        TrialEpoch("trial2", start_idx=14, stop_idx=15),
        TrialEpoch("baseline", start_idx=4, duration=-10),
        TrialEpoch("baseline", start_idx=6, duration=-10),
    ],
    measurements=[m.DurationPerSecond(), m.CountPerSecond()],
    rats=[
        Rat('R155', group="1", gender="male"),
        Rat('R156', group="2", gender="female"),
        Rat('R157', group="2", gender="male"),
        Rat('R158', group="1", gender="female"),
        Rat('R159', group="2", gender="male"),
        Rat('R160', group="1", gender="female"),
        Rat('R161', group="1", gender="male"),
        Rat('R162', group="2", gender="female"),
    ],
    ignore_sessions=['!2017-09-20'],
    sessionfiles=counterconditioning,
)


def add_datapoints(session, data, rat):

    def add_data(cue, trial=None):
        if trial is not None:
            meta = {
                "cue_type": cue[:5],
                "trial_type": trial[-1],
                "rewarded": "rewarded" if trial == "trial1" else "unrewarded",
                "cue": cue,
            }
            trial = data[trial]
            cue = data[cue]
            session.add_epoch_data(rat.rat_id, trial.intersect(cue), meta)
        else:
            meta = {
                "cue_type": cue,
                "trial_type": "",
                "rewarded": "",
                "cue": cue,
            }
            session.add_epoch_data(rat.rat_id, data[cue], meta)

    if rat.group == "1":
        add_data("light1", "trial1")
        add_data("light2", "trial2")
        add_data("baseline")

    elif rat.group == "2":
        add_data("light1", "trial2")
        add_data("light2", "trial1")
        add_data("baseline")


epoch_expt.add_datapoints = add_datapoints
epoch_expt.analyze(cached_data=False)

In [ ]:
epoch_expt.rats[0].rat_id

In [ ]:
for rat in epoch_expt.rats:
    print(rat.rat_id)
    plot_behavior(epoch_expt.df, [rat], filepath=None, by_outcome=True, measure="DurationPerSecond", 
                  labels=["Duration in food cup per second"], colours=probe_colours)
    plot_behavior(epoch_expt.df, [rat], filepath=None, by_outcome=True, measure="CountPerSecond", 
                  labels=["Number of entries per second"], colours=probe_colours)

In [ ]:
epoch_expt = Experiment(
    name="201709",
    cache_key="epoch",
    trial_epochs=[
        TrialEpoch("mags", start_idx=1, stop_idx=2),
        TrialEpoch("light1", start_idx=4, stop_idx=5),
        TrialEpoch("light2", start_idx=6, stop_idx=7),
        TrialEpoch("probe1", start_idx=8, stop_idx=9),
        TrialEpoch("probe2", start_idx=10, stop_idx=11),
        TrialEpoch("trial1", start_idx=12, stop_idx=13),
        TrialEpoch("trial2", start_idx=14, stop_idx=15),
        TrialEpoch("trial3", start_idx=16, stop_idx=17),
        TrialEpoch("trial4", start_idx=18, stop_idx=19),
        TrialEpoch("baseline", start_idx=4, duration=-10),
        TrialEpoch("baseline", start_idx=6, duration=-10),
    ],
    measurements=[m.Duration(), m.DurationPerSecond(), m.CountPerSecond()],
    rats=[
        Rat('R155', group="1", gender="male"),
        Rat('R156', group="2", gender="female"),
        Rat('R157', group="2", gender="male"),
        Rat('R158', group="1", gender="female"),
        Rat('R159', group="2", gender="male"),
        Rat('R160', group="1", gender="female"),
        Rat('R161', group="1", gender="male"),
        Rat('R162', group="2", gender="female"),
    ],
    ignore_sessions=['!2017-09-20'],
    sessionfiles=['!2017-10-16', '!2017-10-17'],
)


def add_datapoints(session, data, rat):

    def add_data(cue, trial=None):
        if trial is not None:
            meta = {
                "cue_type": cue[:5],
                "trial_type": trial[-1],
                "rewarded": "rewarded" if (trial == "trial1") | (trial == "trial3") else "unrewarded",
                "cue": cue,
            }
            trial = data[trial]
            cue = data[cue]
            session.add_epoch_data(rat.rat_id, trial.intersect(cue), meta)
        else:
            meta = {
                "cue_type": cue,
                "trial_type": "",
                "rewarded": "",
                "cue": cue,
            }
            session.add_epoch_data(rat.rat_id, data[cue], meta)

    if rat.group == "1":
        add_data("light1", "trial1")
        add_data("light2", "trial2")
        add_data("probe1", "trial3")
        add_data("probe2", "trial4")
        add_data("baseline")

    elif rat.group == "2":
        add_data("light1", "trial2")
        add_data("light2", "trial1")
        add_data("probe1", "trial4")
        add_data("probe2", "trial3")
        add_data("baseline")


epoch_expt.add_datapoints = add_datapoints
epoch_expt.analyze(cached_data=False)

In [ ]:
for rat in epoch_expt.rats:
    print(rat.rat_id)
    plot_behavior(epoch_expt.df, [rat], filepath=None, by_outcome=True, measure="DurationPerSecond",
                 labels=["Duration in food cup per second"], colours=probe_colours)
    plot_behavior(epoch_expt.df, [rat], filepath=None, by_outcome=True, measure="CountPerSecond",
                 labels=["Number of entries per second"], colours=probe_colours)

In [ ]:
epoch_expt = Experiment(
    name="201709",
    cache_key="epoch",
    trial_epochs=[
        TrialEpoch("mags", start_idx=1, stop_idx=2),
        TrialEpoch("light1", start_idx=5, duration=-10),
        TrialEpoch("light2", start_idx=7, duration=-10),
        TrialEpoch("probe1end", start_idx=9, duration=-10),
        TrialEpoch("probe2end", start_idx=11, duration=-10),
        TrialEpoch("trial1", start_idx=12, stop_idx=13),
        TrialEpoch("trial2", start_idx=14, stop_idx=15),
        TrialEpoch("trial3", start_idx=16, stop_idx=17),
        TrialEpoch("trial4", start_idx=18, stop_idx=19),
        TrialEpoch("baseline", start_idx=4, duration=-10),
        TrialEpoch("baseline", start_idx=6, duration=-10),
    ],
    measurements=[m.Duration(), m.DurationPerSecond(), m.CountPerSecond()],
    rats=[
        Rat('R155', group="1", gender="male"),
        Rat('R156', group="2", gender="female"),
        Rat('R157', group="2", gender="male"),
        Rat('R158', group="1", gender="female"),
        Rat('R159', group="2", gender="male"),
        Rat('R160', group="1", gender="female"),
        Rat('R161', group="1", gender="male"),
        Rat('R162', group="2", gender="female"),
    ],
    ignore_sessions=['!2017-09-20'],
    sessionfiles=probe_sessions,
)


def add_datapoints(session, data, rat):

    def add_data(cue, trial=None):
        if trial is not None:
            meta = {
                "cue_type": cue[:5],
                "trial_type": trial[-1],
                "rewarded": "rewarded" if (trial == "trial1") | (trial == "trial3") else "unrewarded",
                "cue": cue,
            }
            trial = data[trial]
            cue = data[cue]
            session.add_epoch_data(rat.rat_id, trial.intersect(cue), meta)
        else:
            meta = {
                "cue_type": cue,
                "trial_type": "",
                "rewarded": "",
                "cue": cue,
            }
            session.add_epoch_data(rat.rat_id, data[cue], meta)

    if rat.group == "1":
        add_data("light1", "trial1")
        add_data("light2", "trial2")
        add_data("probe1end", "trial3")
        add_data("probe2end", "trial4")
        add_data("baseline")

    elif rat.group == "2":
        add_data("light1", "trial2")
        add_data("light2", "trial1")
        add_data("probe1end", "trial4")
        add_data("probe2end", "trial3")
        add_data("baseline")


epoch_expt.add_datapoints = add_datapoints
epoch_expt.analyze(cached_data=False)

In [ ]:
for rat in epoch_expt.rats:
    print(rat.rat_id)
    plot_behavior(epoch_expt.df, [rat], filepath=None, by_outcome=True, measure="DurationPerSecond",
                 labels=["Duration in food cup per second"], colours=probe_colours)
    plot_behavior(epoch_expt.df, [rat], filepath=None, by_outcome=True, measure="CountPerSecond",
                 labels=["Number of entries per second"], colours=probe_colours)